# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [3]:
sinasc_mun = sinasc_raw[['IDADEMAE', 'IDADEPAI', 'munResNome']]
idademedia_acendentes_por_municipio = sinasc_mun.groupby('munResNome').agg([('média', 'mean')])
idademedia_acendentes_por_municipio.head()

,IDADEMAE,IDADEPAI
,média,média
munResNome,,
Alta Floresta D'Oeste,25.991826,29.548701
Alto Alegre dos Parecis,24.844156,29.179104
Alto Paraíso,24.960177,28.833333
Alvorada D'Oeste,25.770732,30.757282
Ariquemes,25.607866,32.466667


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [4]:
sinasc_raw['DTNASC'].dtype

dtype('O')

In [5]:
sinasc_aniversario = sinasc_raw.loc[:, ('DTNASC', 'PESO', 'SEXO')] #colunas de interesse
sinasc_aniversario['DTNASC'] = pd.to_datetime(sinasc_aniversario['DTNASC']) #conversão em datetime
aniversario_comigo = sinasc_aniversario[sinasc_aniversario['DTNASC'].dt.strftime('%d-%m') == '22-09'].loc[:, ('PESO', 'SEXO')] # filtro
aniversario_comigo.groupby('SEXO').agg({'PESO': [('média', 'mean')]}) #agregação

,PESO
,média
SEXO,
Feminino,3119.000000
Masculino,3285.354839


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [6]:
sinasc_raw['munResNome'].value_counts(ascending=True).to_frame().head(5) # verificando as menores ocorrências de município

,count
munResNome,
Município ignorado - RO,1
Castanheiras,32
Pimenteiras do Oeste,40
Primavera de Rondônia,43
Parecis,44


In [7]:
# há um outlier que vai atrapalhar a análise

# criando o df que será agrupado
sinasc_menos_nasc_mun = sinasc_raw.loc[:,('munResNome', 'IDADEMAE', 'IDADEPAI')]

# removendo cidade ignorada
sinasc_menos_nasc_mun = sinasc_menos_nasc_mun[sinasc_menos_nasc_mun['munResNome'] != 'Município ignorado - RO']

# filtrando para município menos frequente na base
municipio_menor_nasc = sinasc_menos_nasc_mun['munResNome'].value_counts().idxmin() #retorna o indice do .value_counts com menor incidência
print(f'Município com menor quantidade de nascimentos no período: {municipio_menor_nasc}, '
       f'com {sinasc_menos_nasc_mun["munResNome"].value_counts().min()} nascimentos') # mostrando o município e a qtd de nascimentos
sinasc_menos_nasc_mun = sinasc_menos_nasc_mun[sinasc_menos_nasc_mun['munResNome'] == municipio_menor_nasc] #filtra o df de interesse

# agregando

sinasc_menos_nasc_mun.groupby('munResNome').agg([('média', 'mean'),
                                                   ('máximo', 'max'),
                                                   ('mínimo', 'min')])

Município com menor quantidade de nascimentos no período: Castanheiras, com 32 nascimentos


IDADEMAE                 IDADEPAI              
                 média máximo mínimo      média máximo mínimo
munResNome                                                   
Castanheiras  27.28125     39     17  30.392857   43.0   17.0

### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [8]:
# mesma análise do anterior, para maior quantidade de nascimentos

sinasc_mais_nasc_mun = sinasc_raw.loc[:, ('munResNome','QTDFILVIVO', 'IDADEMAE', 'IDADEPAI')]
municipio_maior_nasc = sinasc_mais_nasc_mun['munResNome'].value_counts().idxmax()
sinasc_mais_nasc_mun = sinasc_mais_nasc_mun[sinasc_mais_nasc_mun['munResNome'] == municipio_maior_nasc]
print(f'Município com maior quantidade de nascimentos no período: {municipio_maior_nasc}, '
       f'com {sinasc_mais_nasc_mun["munResNome"].value_counts().max()} nascimentos')
sinasc_mais_nasc_mun.groupby('munResNome').agg([('média', 'mean'),
                                                   ('máximo', 'max'),
                                                   ('mínimo', 'min')])

Município com maior quantidade de nascimentos no período: Porto Velho, com 8437 nascimentos


QTDFILVIVO                IDADEMAE                 IDADEPAI  \
                 média máximo mínimo     média máximo mínimo      média   
munResNome                                                                
Porto Velho   1.167236   12.0    0.0  26.31267     47     12  32.352679   

                           
            máximo mínimo  
munResNome                 
Porto Velho   65.0   16.0

### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

As análises mostram uma maior amplitude nos valores de idades máxima e mínima para os pais em uma amostra maior, ocorrida em uma capital, o que pode indicar questões sociais como subregistro de partos para mães muito novas, o que não seria surpresa em cidades pequenas. As médias são bastante semelhantes, o que, de certa forma, reforça essa observação. Há também maior probabilidade de partos no lar em cidades pequenas, o que reforçaria essa observação